In [17]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [18]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [19]:
# # 1. Read in the AWS S3 bucket into a DataFrame.
# from pyspark import SparkFiles
# url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [20]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [21]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
import time
import urllib.request
# Step 1: Download the CSV file locally
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
local_file_path = "home_sales_revised.csv"
urllib.request.urlretrieve(url, local_file_path)

('home_sales_revised.csv', <http.client.HTTPMessage at 0x1717c89b4c0>)

In [23]:
# Step 2: Read the local CSV file into a DataFrame
df = spark.read.csv(local_file_path, header=True, inferSchema=True)

In [24]:
# Step 3: Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")

In [25]:
# Step 4: Query for the average price of a four-bedroom house sold in each year
avg_price_four_bedrooms = spark.sql("""
    SELECT YEAR(to_date(from_unixtime(date_built))) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(to_date(from_unixtime(date_built)))
    ORDER BY year
""")
avg_price_four_bedrooms.show()

+----+---------+
|year|avg_price|
+----+---------+
|1969|299661.01|
+----+---------+



In [26]:
# Step 5: Query for the average price of a home with three bedrooms and three bathrooms for each year built
avg_price_three_bedrooms_three_bathrooms = spark.sql("""
    SELECT YEAR(to_date(CAST(date_built AS TIMESTAMP))) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(to_date(CAST(date_built AS TIMESTAMP)))
    ORDER BY year
""")

In [27]:
# Step 6: Query for the average price of a home with specific criteria for each year built
avg_price_specific_criteria = spark.sql("""
    SELECT YEAR(to_date(CAST(date_built AS TIMESTAMP))) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(to_date(CAST(date_built AS TIMESTAMP)))
    ORDER BY year
""")

In [28]:
# Step 7: Query for the average price per view rating with an average price >= $350,000
start_time = time.time()
avg_price_per_view_rating = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""")
avg_price_per_view_rating.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

--- 0.13141608238220215 seconds ---


In [29]:
# Step 8: Cache the temporary table and validate the cache
spark.catalog.cacheTable("home_sales")
print(spark.catalog.isCached("home_sales"))

True


In [30]:
# Step 9: Run the query from Step 7 on the cached temporary table and compute the run time
start_time = time.time()
avg_price_per_view_rating_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""")

In [31]:
# Step 10: Write the DataFrame to a Parquet file without partitioning, overwriting if it already exists
df.write.mode("overwrite").parquet("home_sales_partitioned.parquet")

Py4JJavaError: An error occurred while calling o160.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [ ]:
# Step 11: Read the formatted Parquet data into a DataFrame
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")

In [ ]:
# Step 12: Create a temporary view of the Parquet DataFrame
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# Step 13: Run the query from Step 7 on the Parquet temporary table and compute the run time
start_time = time.time()
avg_price_per_view_rating_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM parquet_home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
""")
avg_price_per_view_rating_parquet.show()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Step 14: Uncache the temporary table and validate the cache
spark.catalog.uncacheTable("home_sales")
print(spark.catalog.isCached("home_sales"))